# LecoSpec Data Munging

In [ ]:
# notebooks use their location as their working directory, so
# if we are in a subfolder, move to the main folder.  
# This however can safely be run multiple times
#setwd(M:/lecospec/lecospec)
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

## Notation

Throughout the notebook, variables starting with `img_` are UAV image-based information (data, filepaths, etc).  Similarly, variables beginning with `grd_` related to data collected on the ground.  

Also, some other naming conventions for variables with data transformations:
* `robust` in a variable name refers to data treated by center according to the median and scaling by teh inter-quartile range (a la sklearns RobustScaler)
* `minmax` (and its ilk) are min-max scaled data, i.e. scaled to the interval [0,1] by subtracting the minimum and dividing by the range.
* `standard(ized)` refers to data treated with with the z-score transform by centring using the mean and scaling y the standard deviation (like sklearns StandardScaler)
* `corrected` means that a linear transformation has been applied to account for differences in sensor calibration.
* `raw` refers to having no transformations applied
* `clipped` means that outliers have been clipped to the upper and lower fence values based on the Inter-Quartile Range method. 
* `imputed` means that outliers have been removed and imputed
* `dropped` means that dataframe rows containing outliers have been removed

Example: `img_robust_indices` refers to vegetation indices from the UAV images treated with the robust scaler. 

## Define data locations


In [ ]:
# spectral library
grd_base_path <- "./Output/C_001_SC3_Cleaned_SpectralLib.csv"
grd_speclib <- read.csv(grd_base_path, header = TRUE)
#grd_index_path <- ./Data/D_002_SpecLib_Derivs.csv
#grd_indices <- read.csv(grd_index_path)
# this data has some lines that have no labels, so we remove them 
grd_speclib <- grd_speclib[!is.na(grd_speclib$Functional_group1),]
head(grd_speclib)

In [ ]:
img_base_path <- "Data/Ground_Validation/PFT_Image_spectra/PFT_Image_SpectralLib_Clean.csv"
img_speclib <- read.csv(img_base_path, header = TRUE)
# currently, not using the old pre-proccessing scheme and just doing it here.
#img_index_path <- Data/D_002_Image_SpecLib_Derivs.csv
#img_speclib <- read.csv(img_base_path)
head(img_speclib)

Okay, there are some metadata columns that should not be there for the next step - lets remove them with `subset`

In [ ]:
img_bands <- subset(
    img_speclib, 
    select=-c(
        X,
    	UID,
        ScanNum,
    	sample_name,
    	PFT,
    	FncGrp1
    ))


grd_bands <- subset(
    grd_speclib, 
    select=-c(
        X,
        ScanID,
        Area,
        Code_name,
        Species_name,
        Functional_group1,
        Functional_group2,
        Species_name_Freq,
        Functional_group1_Freq,
        Functional_group2_Freq,
        Genus,
        Version,
        File.Name,
        Instrument,
        Detectors,
        Measurement,
        Date,
        Time,
        Battery.Voltage,
        Averages,
        Integration1,
        Integration2,
        Integration3,
        Dark.Mode,
        Foreoptic,
        Radiometric.Calibration,
        Units,
        Latitude,
        Longitude,
        Altitude,
        GPS.Time,
        Satellites,
        Calibrated.Reference.Correction.File,
        Channels,
        ScanNum
    )
)



Calculate the vegetation indices from the spectral libraries - its easy with lecospectR!

Note that the image-based scpectra are normalized from zero to one, and the ground specctra are on the range zero to one hundred.  

In [ ]:
img_indices <- get_vegetation_indices(img_bands, NULL)# should have a default of NULL, you know?
grd_indices <- get_vegetation_indices(0.01*grd_bands, NULL)

In [ ]:
head(img_indices)
head(img_indices)

This is actually enough to start training models.  We have the vegetation indices, but instead of doing that, let's transform the data and write it to file.  Then we will proceed to creating the model corrections, etc.

In [ ]:
img_resampled_bands <- resample_df(img_bands, drop_existing=TRUE)# corrects scale difference (poorly)
img_resampled_bands <- resample_df(0.01*grd_bands, drop_existing=TRUE)
head(img_resampled_bands)
head(grd_resampled_bands)

In [ ]:
img_raw_with_na <- cbind(img_resampled_bands, img_indices)
grd_raw_with_na <- cbind(grd_resampled_bands, grd_indices)


In [ ]:
img_raw <- impute_spectra(img_raw_with_na)
grd_raw <- impute_spectra(inf_to_na(grd_raw_with_na))# note also dropping an Inf (liekly div by 0 in veg index)

Apply the outlier transforms

In [ ]:
grd_clipped <- clip_outliers(grd_raw)
grd_imputed <- impute_outliers_and_na(grd_raw)
grd_dropped <- grd_raw[detect_outliers_columnwise(grd_raw),]
img_clipped <- clip_outliers(img_raw)
img_imputed <- impute_outliers_and_na(img_raw)
img_dropped <- img_raw[detect_outliers_columnwise(img_raw),]

Now the center/scale transforms

In [ ]:
grd_raw_robust <- columnwise_robust_scale(grd_raw)
img_raw_robust <- columnwise_robust_scale(img_raw)
grd_raw_minmax <- columnwise_min_max_scale(grd_raw)
img_raw_minmax <- columnwise_min_max_scale(img_raw)
grd_raw_standard <- standardize_df(grd_raw)
img_raw_standard <- standardize_df(img_raw)

grd_clipped_robust <- columnwise_robust_scale(grd_clipped)
grd_imputed_robust <- columnwise_robust_scale(grd_imputed)
grd_dropped_robust <- columnwise_robust_scale(grd_dropped)
img_clipped_robust <- columnwise_robust_scale(img_clipped)
img_imputed_robust <- columnwise_robust_scale(img_imputed)
img_dropped_robust <- columnwise_robust_scale(img_dropped)

grd_clipped_minmax <- columnwise_min_max_scale(grd_clipped)
grd_imputed_minmax <- columnwise_min_max_scale(grd_imputed)
grd_dropped_minmax <- columnwise_min_max_scale(grd_dropped)
img_clipped_minmax <- columnwise_min_max_scale(img_clipped)
img_imputed_minmax <- columnwise_min_max_scale(img_imputed)
img_dropped_minmax <- columnwise_min_max_scale(img_dropped)

grd_clipped_standard <- standardize_df(grd_clipped)
grd_imputed_standard <- standardize_df(grd_imputed)
grd_dropped_standard <- standardize_df(grd_imputed)
img_clipped_standard <- standardize_df(img_clipped)
img_imputed_standard <- standardize_df(img_imputed)
img_dropped_standard <- standardize_df(img_dropped)


Now, let's save all these data to disk

In [ ]:
BASE_PATH <- "Data/gs/"
X_TRAIN_PATH <- paste0(BASE_PATH, "x_train/")
Y_TRAIN_PATH <- paste0(BASE_PATH, "y_train/")

X_TEST_PATH <- paste0(BASE_PATH, "x_test/")
Y_TEST_PATH <- paste0(BASE_PATH, "y_test/")

if(!dir.exists(BASE_PATH)){
    dir.create(BASE_PATH)
}
if(!dir.exists(X_TRAIN_PATH)){
    dir.create(X_TRAIN_PATH)
}
if(!dir.exists(Y_TRAIN_PATH)){
    dir.create(Y_TRAIN_PATH)
}
if(!dir.exists(X_TEST_PATH)){
    dir.create(X_TEST_PATH)
}
if(!dir.exists(Y_TEST_PATH)){
    dir.create(Y_TEST_PATH)
}


In [ ]:
write.csv(grd_clipped, file=paste0(X_TRAIN_PATH, "grd_clipped_raw.csv"))
write.csv(grd_clipped_minmax, file=paste0(X_TRAIN_PATH, "grd_clipped_minmax.csv"))
write.csv(grd_clipped_robust, file=paste0(X_TRAIN_PATH, "grd_clipped_robust.csv"))
write.csv(grd_clipped_standard, file=paste0(X_TRAIN_PATH, "grd_clipped_standard.csv"))

write.csv(grd_imputed, file=paste0(X_TRAIN_PATH, "grd_imputed_raw.csv"))
write.csv(grd_imputed_minmax, file=paste0(X_TRAIN_PATH, "grd_imputed_minmax.csv"))
write.csv(grd_imputed_robust, file=paste0(X_TRAIN_PATH, "grd_imputed_robust.csv"))
write.csv(grd_imputed_standard, file=paste0(X_TRAIN_PATH, "grd_imputed_standard.csv"))

write.csv(grd_dropped, file=paste0(X_TRAIN_PATH, "grd_dropped_raw.csv"))
write.csv(grd_dropped_minmax, file=paste0(X_TRAIN_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_dropped_robust, file=paste0(X_TRAIN_PATH, "grd_dropped_robust.csv"))
write.csv(grd_dropped_standard, file=paste0(X_TRAIN_PATH, "grd_dropped_standard.csv"))

write.csv(grd_raw, file=paste0(X_TRAIN_PATH, "grd_raw_raw.csv"))
write.csv(grd_raw_minmax, file=paste0(X_TRAIN_PATH, "grd_raw_minmax.csv"))
write.csv(grd_raw_robust, file=paste0(X_TRAIN_PATH, "grd_raw_robust.csv"))
write.csv(grd_raw_standard, file=paste0(X_TRAIN_PATH, "grd_raw_standard.csv"))

In [ ]:
write.csv(img_clipped, file=paste0(X_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_clipped_minmax, file=paste0(X_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_clipped_robust, file=paste0(X_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_clipped_standard, file=paste0(X_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed, file=paste0(X_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_imputed_minmax, file=paste0(X_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_imputed_robust, file=paste0(X_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_imputed_standard, file=paste0(X_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TRAIN_PATH, "img_dropped_standard.csv"))

write.csv(img_raw, file=paste0(X_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_raw_minmax, file=paste0(X_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_raw_robust, file=paste0(X_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_raw_standard, file=paste0(X_TRAIN_PATH, "img_raw_standard.csv"))

## Labels for the above Data

In [ ]:
img_targets <- img_speclib$FncGrp1 %>% as.factor()
grd_targets <- grd_speclib$Functional_group1 %>% as.factor()

In [ ]:
# drop entries with outliers to match training data
img_targets_dropped <- img_targets[detect_outliers_columnwise(img_raw)]
grd_targets_dropped <- grd_targets[detect_outliers_columnwise(grd_raw)]

In [ ]:
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_standard.csv"))

write.csv(grd_imputed, file=paste0(Y_TRAIN_PATH, "grd_imputed_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_standard.csv"))

write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_standard.csv"))

write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_raw.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_robust.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_standard.csv"))

In [ ]:
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed, file=paste0(Y_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_standard.csv"))

write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_standard.csv"))

## Sensor Correction

In this section, we create the models (and do some data transforms) to make the sensor-correction models and create the corrected data (only three times).  

We do this first for the raw (including outliers) data.

In [ ]:
grd_resampled_to_match_img_bands <- resample_df(
    grd_bands,
    min_wavelength = 398,
    max_wavelength = 999,
    delta=1,
    drop_existing = TRUE
)
head(grd_resampled_to_match_img_bands)
head(img_bands)

In [ ]:
colnames(grd_resampled_to_match_img_bands) <- colnames(img_bands)

In [ ]:
grd_resampled_to_match_img_bands$targets <- grd_targets
img_bands_with_targets <- img_bands
img_bands_with_targets$targets <- img_targets

In [ ]:
matched_data <- create_matched_data(
    img_bands_with_targets,
    grd_resampled_to_match_img_bands,
    cols=c("targets","targets")# assumes joining on columns named "targets" in each data.frame
)
head(matched_data$left)
head(matched_data$right)

In [ ]:
correction_model <- build_columnwise_sensor_correction_model(
    matched_data$left,
    matched_data$right
)

In [ ]:
grd_corrected_bands <- apply_sensor_correction_model(
    correction_model,
    grd_resampled_to_match_img_bands,
    ignore_cols=c("targets")
)
head(grd_corrected_bands)

In [ ]:
grd_corrected_indices <- get_vegetation_indices(grd_corrected_bands, NULL)
head(grd_corrected_indices)

In [ ]:
grd_corrected_resampled_bands <- resample_df(grd_corrected_bands, drop_existing=TRUE)
head(grd_corrected_resampled_bands)

In [ ]:
write.csv(
    cbind(grd_corrected_resampled_bands, grd_corrected_indices), 
    file=paste0(X_TRAIN_PATH, "grd_raw_corrected.csv")
    )

Now that that is done, we will move on to the clipped data

In [73]:
grd_resampled_to_img_clipped <- resample_df(
    clip_outliers(grd_bands),
    min_wavelength = 398,
    max_wavelength = 999,
    delta=1,
    drop_existing = TRUE
)

colnames(grd_resampled_to_img_clipped) <- colnames(img_bands)

grd_resampled_to_img_clipped$targets <- grd_targets
img_bands_with_targets <- img_bands
img_bands_with_targets$targets <- img_targets

matched_data_clipped <- create_matched_data(
    img_bands_with_targets,
    grd_resampled_to_img_clipped,
    cols=c("targets","targets")# assumes joining on columns named "targets" in each data.frame
)

correction_model <- build_columnwise_sensor_correction_model(
    matched_data_clipped$left,
    matched_data_clipped$right
)
grd_corrected_clipped_bands <- apply_sensor_correction_model(
    correction_model,
    grd_resampled_to_match_img_bands,
    ignore_cols=c("targets")
)
grd_corrected_clipped_indices <- get_vegetation_indices(grd_corrected_bands, NULL)
grd_corrected_clipped_resampled_bands <- resample_df(grd_corrected_bands, drop_existing=TRUE)





Call:
lm(formula = left_vec ~ right_vec)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.005867 -0.003500 -0.000347  0.003162  0.034047 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 4.155e-03  2.891e-04   14.37   <2e-16 ***
right_vec   4.009e-03  3.424e-05  117.07   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.004245 on 647 degrees of freedom
Multiple R-squared:  0.9549,	Adjusted R-squared:  0.9549 
F-statistic: 1.371e+04 on 1 and 647 DF,  p-value: < 2.2e-16


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0058369 -0.0032241 -0.0001261  0.0027597  0.0268734 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 4.693e-03  2.715e-04   17.28   <2e-16 ***
right_vec   3.963e-03  3.233e-05  122.59   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual 

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



In [74]:
write.csv(
    cbind(
        grd_corrected_clipped_indices,
        grd_corrected_clipped_resampled_bands
    ),
    file=paste0(
        X_TRAIN_PATH,
        "grd_clipped_corrected.csv"
    )
)

And finally the dropped outlier one

notes for later: should probably try PCA here.  clip -> scale -> PCA -> subset (and scale again for models like SVM and kNN)

...to be continued